# Projeto 1 - Ciência dos Dados

Nome: Vitor Fortes Giuliano Riccetti 

Nome: Rodrigo Paoliello de Medeiros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'Chuteira_Puma.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Chuteira_Puma.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
# Faz a leitura do excel 
train = pd.read_excel(filename)
train.head(5)

train["Relevante"]= train["Relevante"].astype(str)
train["Irrelevante"]= train["Irrelevante"].astype(str)



In [4]:
test = pd.read_excel(filename, sheet_name = 'Treinamento')
test.head(5)

,Treinamento,Relevante,Irrelevante
0,NaN,"@puma_br @palmeiras meu time, nossa cria, noss...",@neymarjr @puma chamou na chuteira clássica
1,NaN,doido que as pelada na arena volta pra eu lanç...,"porra ney, era pra inaugurar a chuteira da pum..."
2,NaN,essa chuteira da puma😍😍😍 https://t.co/ra7ojthqxs,@neymarjr @puma chuteira preta! ja posso iludi...
3,NaN,@neymarjr @puma tem uma chuteira mais bonitinh...,"bicho, o neymar recusou renovação de contrato ..."
4,NaN,@gaboquesabe nunca vi uma chuteira top tier 1 ...,@neymarjr @puma chuteira preta pai agora a cop...


In [5]:
# dividimos em duas categorias relevante e irrelevante:

Relevantes = ''.join(train.Relevante)

Irrelevantes = ' '.join(train.Irrelevante)


In [6]:
# Função para limpar a string
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    text = re.sub(r'@\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'kk\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;_“”]' # Note que os sinais [] são delimitadores de um conjunto.

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
shrink_whitespace_reg = re.compile(r'\s{2,}')

def clean_text(text_subbed):
    reg = re.compile(r'({})'.format(emoji_pat)) # line a
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', text_subbed)
    return shrink_whitespace_reg.sub(' ', result)




In [7]:
# Retira as pontuações e os links

Lista_R_clean = cleanup(Relevantes.lower())
Lista_R_clean = clean_text(Lista_R_clean)

Lista_R_clean

' meu time nossa cria nossa poderia me presentear com uma chuteira pra fechar o squad rsrsrs boyah doido que as pelada na arena volta pra eu lançar uma chuteira da puma igual menino ney 😅 essa chuteira da puma 😍 😍 😍 tem uma chuteira mais bonitinha ñ ney nunca vi uma chuteira top tier 1 da puma neymar assinando com a puma e lançando uma chuteira horrorosa essa chuteira preta básica da puma é clássica dmsss eu falo em chuteira mesmo já testei adidas puma mas a nike é a única que encaixa perfeitamente chuteira feia acabando com meu sonho de ter uma chuteira puma agora vai custar 2 mil reais opinião chuteira da puma é muito fatela em termos de tudonem com o dinheiro q o ney ganha da puma eu usaria essa chuteira feia vsf tô apaixonada nessa chuteira preta da puma do neymar 🤤🤤chuteira preta ue ate eu vou usar puma agr 😂 a chuteira a da puma é superior a todas eles tem o modelo de chuteiras mais leve do mundoisso é fato ele é o neymar né a gente bate o olho em um produto que ele está usando e

In [8]:
# Retira as pontuações e os links

Lista_IR_clean = cleanup(Irrelevantes.lower())
Lista_IR_clean = clean_text(Lista_IR_clean)

Lista_IR_clean


' chamou na chuteira clássica porra ney era pra inaugurar a chuteira da puma na cara dele 🔥 chuteira preta ja posso iludir ou tá cedo bicho o neymar recusou renovação de contrato com a nike porque a nike fez uma chuteira do mbape e não fez pra ele e por isso ele assinou com a puma ahkdhdjshdjsbs chuteira preta pai agora a copa vem dobrou a meia pra n aparecer a nike perto da chuteira a chuteira preta senti o impacto neymar de chuteira da puma é estranho mas amei na hora q sair uma chuteira da puma que seje do ney da cor de um canetão aí sim vai ser a cara dele neymar devia te enfiado a chuteira da puma na bunda desse mlk neymar com essa chuteira preta da puma = hexa chuteira preta já avisto o hexa em 2022 esperando o lançar uma chuteira doida da neymar de chuteira da puma da até arrepios pai tá on e de chuteira preta da puma chuteira preta = hexa já botei o neymar com a chuteira da puma aq no fifa caralho neymar calçou uma chuteira puma e tudo mudou chuteira preta to iludido já neymar 

In [9]:
# Serie de palavras Relevantes
Palavras_relevantes = pd.Series(Lista_R_clean.split())

len(Palavras_relevantes)

1759

In [10]:
# Serie de palavras Irrelevantes
Palavras_irrelevantes = pd.Series(Lista_IR_clean.split())

len(Palavras_irrelevantes)

2105

In [11]:
# Tabela absoluta dos Relevantes.
tabela_absoluta_relevantes = Palavras_relevantes.value_counts(ascending=False)


tabela_absoluta_relevantes.head(30)

chuteira    137
puma        124
da           99
a            51
que          42
é            38
uma          32
do           26
essa         25
de           25
e            25
o            23
eu           23
preta        22
neymar       19
não          19
comprar      16
com          16
vou          15
mas          15
pra          14
feia         13
um           12
ney          12
só           11
q            11
vai          11
nike         10
tem          10
mais         10
dtype: int64

In [12]:
# Tabela absoluta dos Irrelevantes.
tabela_absoluta_irrelevantes = Palavras_irrelevantes.value_counts(ascending=False)


tabela_absoluta_irrelevantes.head(30)

chuteira    154
da          125
puma        113
a            83
neymar       56
o            55
preta        42
e            42
com          42
de           38
do           33
uma          28
que          26
não          21
é            21
nike         20
pra          19
hexa         17
ney          17
vai          17
ele          16
na           14
tá           13
q            13
cara         13
essa         13
agora        11
um           11
eu           11
só           11
dtype: int64

In [13]:
# Frequencia relativa dos Relevantes 

tabela_relativa_relevantes = Palavras_relevantes.value_counts(True)

tabela_relativa_relevantes

chuteira    0.077885
puma        0.070495
da          0.056282
a           0.028994
que         0.023877
              ...   
quase       0.000569
sai         0.000569
curto       0.000569
juntando    0.000569
pow         0.000569
Length: 561, dtype: float64

In [14]:
tabela_relativa_relevantes.sum()

1.0

In [15]:
# Frequencia relativa dos Irrelevantes

tabela_relativa_irrelevantes = Palavras_irrelevantes.value_counts(True)

tabela_relativa_irrelevantes

chuteira    0.073159
da          0.059382
puma        0.053682
a           0.039430
neymar      0.026603
              ...   
bêbado      0.000475
queria      0.000475
viadagem    0.000475
comprou     0.000475
😜           0.000475
Length: 613, dtype: float64

In [16]:
tabela_relativa_irrelevantes.sum()

0.9999999999999999

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [17]:
total = len(Palavras_relevantes) + len(Palavras_irrelevantes)
total_relevantes = len(Palavras_relevantes)/total
total_irrelevantes = len(Palavras_irrelevantes)/total
print(total_relevantes)
print(total_irrelevantes)

0.4552277432712215
0.5447722567287785


In [25]:
#palavras que nos selecionamos 

lista_palavras = ['chuteira','bonita','feia','quero','comprar','não','gostei','ruim','boa','puma','nike','adidas','melhor','neymar','estranho']


In [26]:
#probabilidade das palavras estarem como relevantes 

chuteira_r = (tabela_absoluta_relevantes['chuteira'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
bonita_r = (tabela_absoluta_relevantes['bonita'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
feia_r = (tabela_absoluta_relevantes['feia'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
quero_r = (tabela_absoluta_relevantes['quero'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
comprar_r = (tabela_absoluta_relevantes['comprar'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
nao_r = (tabela_absoluta_relevantes['não'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
gostei_r = (tabela_absoluta_relevantes['gostei'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
ruim_r = (tabela_absoluta_relevantes['ruim'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
boa_r = (tabela_absoluta_relevantes['boa'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
puma_r = (tabela_absoluta_relevantes['puma'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
nike_r = (tabela_absoluta_relevantes['nike'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
adidas_r = (tabela_absoluta_relevantes['adidas'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
melhor_r = (tabela_absoluta_relevantes['melhor'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
neymar_r = (tabela_absoluta_relevantes['neymar'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
estranho_r = (tabela_absoluta_relevantes['estranho'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))

In [27]:
#probabilidade das palavras estarem como irrelevantes 

chuteira_ir = (tabela_absoluta_irrelevantes['chuteira'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
bonita_ir = (tabela_absoluta_irrelevantes['bonita'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
feia_ir = (tabela_absoluta_irrelevantes['feia'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
quero_ir = (tabela_absoluta_irrelevantes['quero'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
comprar_ir = (tabela_absoluta_irrelevantes['comprar'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
nao_ir = (tabela_absoluta_irrelevantes['não'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
gostei_ir = (tabela_absoluta_irrelevantes['gostei'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
ruim_ir = (0 + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
boa_ir = (tabela_absoluta_irrelevantes['boa'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
puma_ir = (tabela_absoluta_irrelevantes['puma'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
nike_ir = (tabela_absoluta_irrelevantes['nike'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
adidas_ir = (tabela_absoluta_irrelevantes['adidas'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
melhor_ir = (tabela_absoluta_irrelevantes['melhor'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
neymar_ir = (tabela_absoluta_irrelevantes['neymar'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
estranho_ir = (tabela_absoluta_irrelevantes['estranho'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))

In [28]:
P_chuteira_bonita_r = chuteira_r * bonita_r * total_relevantes
P_chuteira_bonita_r

0.0001397328822055775

In [29]:
P_chuteira_bonita_ir = chuteira_ir * bonita_ir * total_irrelevantes
P_chuteira_bonita_ir

3.757551610580307e-05

In [30]:
P_chuteira_feia_r = chuteira_r * feia_r * total_relevantes
P_chuteira_feia_r

0.000279465764411155

In [31]:
P_chuteira_feia_ir = chuteira_ir * feia_ir * total_irrelevantes
P_chuteira_feia_ir

3.757551610580307e-05

In [32]:
P_quero_comprar_chuteira_r = quero_r * comprar_r * chuteira_r * total_relevantes
P_quero_comprar_chuteira_r

5.73874777942056e-07

In [33]:
P_quero_comprar_chuteira_ir = quero_ir * comprar_ir * chuteira_ir * total_irrelevantes
P_quero_comprar_chuteira_ir

1.0634580029944266e-07

In [34]:
P_nike_melhor_r = nike_r * melhor_r * total_relevantes
P_nike_melhor_r

4.773483553606684e-06

In [35]:
P_nike_melhor_ir = nike_ir * melhor_ir * total_irrelevantes
P_nike_melhor_ir

1.0181752751249865e-05

In [36]:
P_adidas_melhor_r = adidas_r * melhor_r * total_relevantes
P_adidas_melhor_r

1.3018591509836412e-06

In [37]:
P_adidas_melhor_ir = adidas_ir * melhor_ir * total_irrelevantes
P_adidas_melhor_ir

1.939381476428546e-06

In [38]:
P_neymar_puma_estranho_r = neymar_r * puma_r * estranho_r * total_relevantes
P_neymar_puma_estranho_r

4.0769734153314575e-07

In [39]:
P_neymar_puma_estranho_ir = neymar_ir * puma_ir * estranho_ir * total_irrelevantes
P_neymar_puma_estranho_ir

2.2291451946638017e-06

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [40]:
test.head()

,Treinamento,Relevante,Irrelevante
0,NaN,"@puma_br @palmeiras meu time, nossa cria, noss...",@neymarjr @puma chamou na chuteira clássica
1,NaN,doido que as pelada na arena volta pra eu lanç...,"porra ney, era pra inaugurar a chuteira da pum..."
2,NaN,essa chuteira da puma😍😍😍 https://t.co/ra7ojthqxs,@neymarjr @puma chuteira preta! ja posso iludi...
3,NaN,@neymarjr @puma tem uma chuteira mais bonitinh...,"bicho, o neymar recusou renovação de contrato ..."
4,NaN,@gaboquesabe nunca vi uma chuteira top tier 1 ...,@neymarjr @puma chuteira preta pai agora a cop...


In [42]:
frase_test = []
r = 1
i = 1
clas = []
for x in test.Relevante:
    print(x)
    #x *=
    #i *=
    for p in x.split():
        print(p)
        
    
    
    
#R= total*r
#I= total*i

@puma_br @palmeiras meu time, nossa cria, nossa @puma_br poderia me presentear com uma chuteira pra fechar o squad! rsrsrs boyah!
@puma_br
@palmeiras
meu
time,
nossa
cria,
nossa
@puma_br
poderia
me
presentear
com
uma
chuteira
pra
fechar
o
squad!
rsrsrs
boyah!
doido que as pelada na arena volta pra eu lançar uma chuteira da puma igual menino ney 😅
doido
que
as
pelada
na
arena
volta
pra
eu
lançar
uma
chuteira
da
puma
igual
menino
ney
😅
essa chuteira da puma😍😍😍 https://t.co/ra7ojthqxs
essa
chuteira
da
puma😍😍😍
https://t.co/ra7ojthqxs
@neymarjr @puma tem uma chuteira mais bonitinha ñ ney?
@neymarjr
@puma
tem
uma
chuteira
mais
bonitinha
ñ
ney?
@gaboquesabe nunca vi uma chuteira top tier 1 da puma.
@gaboquesabe
nunca
vi
uma
chuteira
top
tier
1
da
puma.
neymar assinando com a puma e lançando uma chuteira horrorosa...
neymar
assinando
com
a
puma
e
lançando
uma
chuteira
horrorosa...
essa chuteira preta básica da puma é clássica dmsss! https://t.co/6oqjpwsssy
essa
chuteira
preta
básica
da
puma
é


agora
com
o
@neymarjr
vai
ficar
outro
patmar
ta
maluco
🔥🔥
@neymarjr @puma chuteira feia da porra!
@neymarjr
@puma
chuteira
feia
da
porra!
@neymarjr @puma essa chuteira ficou braba
@neymarjr
@puma
essa
chuteira
ficou
braba
@jfpeixot0 ah mané, não é por nada não mas minha próxima chuteira é puma ff
@jfpeixot0
ah
mané,
não
é
por
nada
não
mas
minha
próxima
chuteira
é
puma
ff
era só essa chuteira, puma 🤤❤️ https://t.co/5motf1swoy
era
só
essa
chuteira,
puma
🤤❤️
https://t.co/5motf1swoy
melhor coisa que ele fez pprt, será que o pai influência ?????? chuteira da puma é surreal de boa https://t.co/vqzq41hklm
melhor
coisa
que
ele
fez
pprt,
será
que
o
pai
influência
??????
chuteira
da
puma
é
surreal
de
boa
https://t.co/vqzq41hklm
essa chuteira nova do neymar da puma tá braba. sempre gostei de chuteira toda preta
essa
chuteira
nova
do
neymar
da
puma
tá
braba.
sempre
gostei
de
chuteira
toda
preta
será que essa é a única chuteira neurótica da puma? https://t.co/xxvxoehw7j
será
que
essa
é
a
única
chut

AttributeError: 'float' object has no attribute 'split'

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**